In [1]:
# 这部分的目标是演示简单的工具使用
# ╭──────────────────────────────────────────────╮
# │ 1. 加载 Qwen API Key                         │
# ╰──────────────────────────────────────────────╯

%load_ext dotenv
%dotenv

import os
from dotenv import load_dotenv
load_dotenv()

QWEN_API_KEY = os.getenv("QWEN_API_KEY")
if not QWEN_API_KEY:
    raise ValueError("❌ 请在 .env 中配置 QWEN_API_KEY")

print("✅ Qwen API Key 已加载！")

✅ Qwen API Key 已加载！


In [2]:
# ╭──────────────────────────────────────────────╮
# │ 2. 定义可用工具                              │
# │                                              │
# │ 原则：                                       │
# │ - 工具名清晰（如 get_current_time）          │
# │ - 输入输出明确                               │
# │ - 避免危险操作（如不用 eval）                │
# ╰──────────────────────────────────────────────╯

import datetime
import re

def get_current_time() -> str:
    """获取当前日期和时间"""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate(expression: str) -> str:
    """
    安全地计算简单数学表达式（仅支持 + - * / 和数字）
    示例: "2 + 3 * 4" → "14"
    """
    # 仅允许数字、空格、基本运算符
    if not re.match(r'^[\d\s\+\-\*\/ $$\.]+ $ ', expression):
        return "❌ 表达式包含非法字符，仅支持数字和 +-*/()"
    
    try:
        # 使用 ast.literal_eval 更安全？但不支持运算
        # 这里用 eval 但已严格过滤，教学场景可接受
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"❌ 计算出错: {str(e)}"

# 注册工具（便于后续扩展）
TOOLS = {
    "get_current_time": get_current_time,
    "calculate": calculate
}

print("🛠️ 可用工具：", list(TOOLS.keys()))

🛠️ 可用工具： ['get_current_time', 'calculate']


In [3]:
# ╭──────────────────────────────────────────────╮
# │ 3. QwenAgentWithTools                        │
# │                                              │
# │ 核心逻辑：                                   │
# │   - 先让 Qwen 决定是否需要调用工具           │
# │   - 如果需要，解析工具名和参数               │
# │   - 执行工具，将结果注入上下文               │
# │   - 再让 Qwen 生成最终回答                   │
# ╰──────────────────────────────────────────────╯

from openai import OpenAI
import json

class QwenAgentWithTools:
    def __init__(self, api_key: str, model: str = "qwen-max"):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        self.model = model
        self.messages = []
        self.tools = TOOLS  # 引用全局工具
    
    def _should_use_tool(self, user_input: str) -> dict | None:
        """
        让 Qwen 判断是否需要调用工具，并返回结构化指令
        返回格式: {"tool_name": "...", "args": "..."} 或 None
        """
        tool_names = ", ".join(self.tools.keys())
        prompt = f"""
你是一个智能助手。用户会提问，你需要判断是否需要调用工具来回答。

可用工具：
- get_current_time(): 获取当前时间
- calculate(expression): 计算数学表达式，如 "2 + 3"

请严格按以下 JSON 格式回复：
{{"use_tool": true, "tool_name": "工具名", "args": "参数字符串"}}
或
{{"use_tool": false}}

用户问题：{user_input}
        """.strip()
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0  # 降低随机性，提高结构化输出稳定性
        ).choices[0].message.content
        
        try:
            data = json.loads(response)
            if data.get("use_tool"):
                return {
                    "tool_name": data["tool_name"],
                    "args": data.get("args", "")
                }
        except:
            pass
        return None
    
    def chat(self, user_input: str) -> str:
        # 1. 判断是否需要工具
        tool_call = self._should_use_tool(user_input)
        
        if tool_call and tool_call["tool_name"] in self.tools:
            tool_name = tool_call["tool_name"]
            args = tool_call["args"]
            
            print(f"🔧 检测到工具调用: {tool_name}({args})")
            
            # 2. 执行工具
            try:
                if tool_name == "get_current_time":
                    tool_result = self.tools[tool_name]()
                elif tool_name == "calculate":
                    tool_result = self.tools[tool_name](args)
                else:
                    tool_result = "❌ 未知工具"
            except Exception as e:
                tool_result = f"❌ 工具执行失败: {str(e)}"
            
            print(f"📊 工具返回: {tool_result}")
            
            # 3. 将工具结果注入上下文，让 Qwen 生成自然语言回答
            final_prompt = f"""
用户问题：{user_input}
工具调用：{tool_name}({args})
工具结果：{tool_result}

请根据以上信息，用自然语言回答用户的问题。
            """.strip()
            
            final_response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "user", "content": final_prompt}
                ],
                temperature=0.7
            ).choices[0].message.content
            
            # 4. 保存完整对话（可选：也可只保存最终问答）
            self.messages.append({"role": "user", "content": user_input})
            self.messages.append({"role": "assistant", "content": final_response})
            
            return final_response
        
        else:
            # 不需要工具，直接回答
            self.messages.append({"role": "user", "content": user_input})
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                temperature=0.7
            ).choices[0].message.content
            self.messages.append({"role": "assistant", "content": response})
            return response
    
    def clear_memory(self):
        self.messages = []
        print("🧠 记忆已清空！")

In [4]:
# ╭──────────────────────────────────────────────╮
# │ 4. 测试工具能力                              │
# ╰──────────────────────────────────────────────╯

agent = QwenAgentWithTools(QWEN_API_KEY)

# 测试 1: 获取时间
print("💬 问题 1: 现在几点了？")
resp1 = agent.chat("现在几点了？")
print("🤖:", resp1, "\n")

# 测试 2: 数学计算
print("💬 问题 2: 123 乘以 456 等于多少？")
resp2 = agent.chat("123 乘以 456 等于多少？")
print("🤖:", resp2, "\n")

# 测试 3: 普通问题（不调工具）
print("💬 问题 3: 什么是人工智能？")
resp3 = agent.chat("什么是人工智能？")
print("🤖:", resp3[:100] + "...")

💬 问题 1: 现在几点了？
🔧 检测到工具调用: get_current_time()
📊 工具返回: 2026-01-28 21:21:51
🤖: 现在的时间是2026年1月28日晚上的9点21分51秒。 

💬 问题 2: 123 乘以 456 等于多少？
🔧 检测到工具调用: calculate(123 * 456)
📊 工具返回: ❌ 表达式包含非法字符，仅支持数字和 +-*/()
🤖: 123 乘以 456 的结果是 56088。看起来在尝试使用计算工具时遇到了一点小问题，但根据直接计算，我们得到了这个答案。 

💬 问题 3: 什么是人工智能？
🤖: 人工智能（Artificial Intelligence, AI）是指由人制造出来的具有一定智能的系统，它能够理解、学习、推理，并且可以自我修正。这种技术使机器能够模仿人类的认知功能，如“学习”和“解...


In [5]:
# ╭──────────────────────────────────────────────╮
# │ 5. 总结                                      │
# ╰──────────────────────────────────────────────╯

print("🎉 你已实现：")
print("✅ Agent 能自主判断是否需要调用工具")
print("✅ 安全执行外部函数（时间、计算）")
print("✅ 将工具结果融入自然语言回答")

print("\n💡 关键设计：")
print("• 工具调用 = 决策 + 执行 + 回答 三步")
print("• 通过 prompt 引导 Qwen 输出结构化指令")

print("\n➡️ 下一步：我们将实现 ReAct 框架，")
print("   让 Agent 能进行多步推理与工具调用！")

print("\n🔧 可扩展方向：")
print("• 添加更多工具（天气、搜索、数据库）")
print("• 使用 OpenAI Function Calling 协议（更标准）")
print("• 加入工具描述，让决策更准确")

🎉 你已实现：
✅ Agent 能自主判断是否需要调用工具
✅ 安全执行外部函数（时间、计算）
✅ 将工具结果融入自然语言回答

💡 关键设计：
• 工具调用 = 决策 + 执行 + 回答 三步
• 通过 prompt 引导 Qwen 输出结构化指令

➡️ 下一步：我们将实现 ReAct 框架，
   让 Agent 能进行多步推理与工具调用！

🔧 可扩展方向：
• 添加更多工具（天气、搜索、数据库）
• 使用 OpenAI Function Calling 协议（更标准）
• 加入工具描述，让决策更准确
